Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat_v2(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat_v2(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.294413 learning rate: 10.000000
Minibatch perplexity: 26.96
dz ikfe  g    aewxiirbcanviy collsusqotzvozmetjw cyipuhz yreeoe rjrcho rdmswgwtc
golsidabegaedzoeqnmbhpzfnubg mpkj g lddidto t pte lteee na jpxkatydeijrlnooulk p
v wcrppeorhkaxz stuaneveviagb ed  lfrloeq dem cssnamauv dl ntutui indido l bzrxi
sapteipbjeso mn in onuql chcc  qcgeed rbo  yedoibshh  ilkiot wnempem qxfoezm qld
dalovu ddwtreiqe dwvdhudw bbfip genuglipfgufvt ormewfkalxxl aiqiljgnwiez mgaa w 
Validation set perplexity: 20.27
Average loss at step 100: 2.589133 learning rate: 10.000000
Minibatch perplexity: 11.01
Validation set perplexity: 10.26
Average loss at step 200: 2.239149 learning rate: 10.000000
Minibatch perplexity: 8.48
Validation set perplexity: 8.62
Average loss at step 300: 2.101283 learning rate: 10.000000
Minibatch perplexity: 7.47
Validation set perplexity: 8.05
Average loss at step 400: 2.005828 learning rate: 10.000000
Minibatch perplexity: 7.52
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.


In [10]:
# LSTM weights/biases, e.g. w_lstm_input, accounts for four of the previous variables, e.g. ix, fx, cx, ox.
# LSTM cell (lstm_cell) now only has two matrix multiplications.

num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # LSTM weights/biases
  w_lstm_input = tf.Variable(tf.truncated_normal([vocabulary_size, 4 * num_nodes], -0.1, 0.1))
  b_lstm = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  w_lstm_output = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    p = tf.matmul(i, w_lstm_input) + tf.matmul(o, w_lstm_output) + b_lstm
    input_gate = tf.sigmoid(p[:, :num_nodes])
    forget_gate = tf.sigmoid(p[:, num_nodes:(2 * num_nodes)])
    update = p[:, (2 * num_nodes):(3 * num_nodes)]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(p[:, (3 * num_nodes):])
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat_v2(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat_v2(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

In [11]:
num_nodes = 64
embedding_size = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  keep_prob=tf.placeholder(tf.float32)
  # Bigram embeddings
  embed = tf.Variable(tf.random_uniform([vocabulary_size * vocabulary_size, embedding_size], -0.1, 0.1))
  # LSTM weights/biases
  w_lstm_input = tf.Variable(tf.truncated_normal([embedding_size, 4 * num_nodes], -0.1, 0.1))
  b_lstm = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  w_lstm_output = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """LSTM cell with fewer matrix multiplies."""
    p = tf.matmul(tf.nn.dropout(i, keep_prob), w_lstm_input) + tf.matmul(o, w_lstm_output) + b_lstm
    input_gate = tf.sigmoid(p[:, :num_nodes])
    forget_gate = tf.sigmoid(p[:, num_nodes:(2 * num_nodes)])
    update = p[:, (2 * num_nodes):(3 * num_nodes)]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(p[:, (3 * num_nodes):])
    output = tf.nn.dropout(output_gate * tf.tanh(state), keep_prob)
    return output, state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size, vocabulary_size]))
  train_inputs = list()
  for i in range(num_unrollings - 1):
    train_inputs.append(tf.nn.embedding_lookup(embed, tf.argmax(train_data[i], 1) * vocabulary_size + tf.argmax(train_data[i + 1], 1)))
  train_labels = train_data[2:]

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat_v2(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat_v2(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[2, vocabulary_size])
  sample_input_embed = tf.expand_dims(tf.nn.embedding_lookup(embed, tf.argmax(sample_input[0], 0) * vocabulary_size + tf.argmax(sample_input[1], 0)), 0)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input_embed, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [12]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = {keep_prob: 0.5}
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[2:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = np.array([sample(random_distribution())[0] for _ in range(2)])
          sentence = ''.join(characters(feed))
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
            feed = np.array([feed[1], sample(prediction)[0]])
            sentence += characters(feed)[1]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      b = valid_batches.next()
      for _ in range(valid_size):
        predictions = sample_prediction.eval({sample_input: np.concatenate(b), keep_prob: 1.0})
        b = valid_batches.next()
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.299389 learning rate: 10.000000
Minibatch perplexity: 27.10
flaeb qvaki aim lx  srr rjeuweqrregtkna jel   zscbhejutlztd e t  lzgni pdtacst qi
mwimg ai tue  dolstgo  mxfhyp z mge oifv yeaikn ioyos srhhtsgimkhfbc ripumobm uqe
jdrq efh ujg rpduehev szzirhhhto clixaisleknao lfc swvqi uy czhxs clegghhn  shg r
lcyix r yer ych ferorihllibiu tetknnnxtclmgq oe mg o un   uj  sxagalu jy wyesgen 
hozeteycabn icez bzraqy be pavzwzdqmyhpcrvelr  ub dvjytabplmgqe unnwrhveimnhssjtn
Validation set perplexity: 20.16
Average loss at step 100: 2.630300 learning rate: 10.000000
Minibatch perplexity: 11.70
Validation set perplexity: 10.84
Average loss at step 200: 2.343119 learning rate: 10.000000
Minibatch perplexity: 10.78
Validation set perplexity: 9.28
Average loss at step 300: 2.264440 learning rate: 10.000000
Minibatch perplexity: 7.94
Validation set perplexity: 8.63
Average loss at step 400: 2.233430 learning rate: 10.000000
Minibatch perplexity: 10.41
Validation 

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

Below is a sequence-to-sequence LSTM model which accepts a sequence of variable length (less that max_seq_len) and outputs the sequence in reverse order. We use sequences of digits instead of characters for simplicity, though it is just as general.

This model features an encoder and decoder, each with their own parameters. The encoder encodes the input sequence and feeds its result to the decoder, which accepts the encoding and outputs the sequence in reverse order. In order to handle variable sequence lengths, the model uses padding and bucketing during its training phase.

Some ways to improve this model:
- Use tensorflow's built-in dynamic_rnn cells. I purposely avoided these to challenge myself to build the model from 'scratch.'
- Use the 'attention' mechanism.
- Use multi-layered LSTMs (computationally slower).




In [13]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import tensorflow as tf

In [18]:
vocab_size = 10
max_seq_len = 5

num_nodes = 32
batch_size = 64

graph = tf.Graph()
with graph.as_default():
  
    # Parameters:
    # Encoder LSTM weights/biases.
    w_enc = tf.Variable(tf.truncated_normal([vocab_size + 1 + num_nodes, 4 * num_nodes], -0.1, 0.1))
    b_enc = tf.Variable(tf.constant(0.1, shape=[1, 4 * num_nodes]))
    # Decoder LSTM weights/biases.
    w_dec = tf.Variable(tf.truncated_normal([vocab_size + 1 + num_nodes, 4 * num_nodes], -0.1, 0.1))
    b_dec = tf.Variable(tf.constant(0.1, shape=[1, 4 * num_nodes]))
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocab_size + 1], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocab_size + 1]))
    
    # Encoder LSTM.
    def enc(i, output, state):
        p = tf.matmul(tf.concat_v2([i, output], 1), w_enc) + b_enc
        input_gate = tf.sigmoid(p[:, :num_nodes])
        forget_gate = tf.sigmoid(p[:, num_nodes:(2 * num_nodes)])
        update = p[:, (2 * num_nodes):(3 * num_nodes)]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(p[:, (3 * num_nodes):])
        output = output_gate * tf.tanh(state)
        return output, state

    # Decoder LSTM
    def dec(i, output, state):
        p = tf.matmul(tf.concat_v2([i, output], 1), w_dec) + b_dec
        input_gate = tf.sigmoid(p[:, :num_nodes])
        forget_gate = tf.sigmoid(p[:, num_nodes:(2 * num_nodes)])
        update = p[:, (2 * num_nodes):(3 * num_nodes)]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(p[:, (3 * num_nodes):])
        output = output_gate * tf.tanh(state)
        return output, state
    
    # Train data.
    train_inputs = list()
    train_labels = list()
    for _ in range(max_seq_len):
        train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size, vocab_size + 1]))
        train_labels.append(tf.placeholder(tf.float32, shape=[batch_size, vocab_size + 1]))
    train_eos = [tf.one_hot(tf.constant(vocab_size, shape=[batch_size]), vocab_size + 1)]

    # Unrolled Encoder LSTM loop.
    enc_output = tf.zeros([batch_size, num_nodes])
    enc_state = tf.zeros([batch_size, num_nodes])
    for i in train_inputs:
        enc_output, enc_state = enc(i, enc_output, enc_state)
    
    # Unrolled Decoder LSTM loop.    
    outputs = list()
    dec_output = enc_output
    dec_state = tf.zeros([batch_size, num_nodes])
    for i in (train_eos + train_labels):
        dec_output, dec_state = dec(i, dec_output, dec_state)
        outputs.append(tf.matmul(dec_output, w) + b)

    # Loss
    logits = tf.concat_v2(outputs, 0)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat_v2(train_labels + train_eos, 0))) #+ 0.01 * (tf.nn.l2_loss(w_enc) + tf.nn.l2_loss(w_dec) + tf.nn.l2_loss(w))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.01, global_step, 5000 * max_seq_len, 0.1, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # Sampling.
    sample_i = tf.placeholder(tf.float32, shape=[1, vocab_size + 1])
    saved_sample_enc_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_enc_state = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_dec_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_dec_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample = tf.group(saved_sample_enc_output.assign(tf.zeros([1, num_nodes])),
                            saved_sample_enc_state.assign(tf.zeros([1, num_nodes])), 
                            saved_sample_dec_output.assign(tf.zeros([1, num_nodes])),
                            saved_sample_dec_state.assign(tf.zeros([1, num_nodes])))
    init_dec = tf.group(saved_sample_dec_output.assign(saved_sample_enc_output))
    
    # Encode sample.
    sample_enc_output, sample_enc_state = enc(sample_i, saved_sample_enc_output, saved_sample_enc_state)
    train_enc = tf.group(saved_sample_enc_output.assign(sample_enc_output),
                         saved_sample_enc_state.assign(sample_enc_state))
    
    # Decode sample.
    sample_dec_output, sample_dec_state = dec(sample_i, saved_sample_dec_output, saved_sample_dec_state)
    with tf.control_dependencies([saved_sample_dec_output.assign(sample_dec_output),
                                  saved_sample_dec_state.assign(sample_dec_state)]):
        sample_prediction = tf.nn.softmax(tf.matmul(sample_dec_output, w) + b)
        

In [20]:
num_steps = 7001
summary_frequency = 1000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    
    for step in range(num_steps):
        
        # At each step, train model with a batch for every sequence length.
        for seq_len in range(1, max_seq_len + 1):
            feed_dict = dict()
            
            # Random seq_len number of digits,
            for i in range(seq_len):
                feed = (np.arange(vocab_size + 1) == np.random.randint(vocab_size, size=batch_size)[:, None]).astype(np.float)
                feed_dict[train_inputs[i]] = feed
                feed_dict[train_labels[seq_len - 1 - i]] = feed
                
            # Padded with EOS.
            for i in range(seq_len, max_seq_len):
                feed_dict[train_inputs[i]] = (np.arange(vocab_size + 1) == np.full(batch_size, vocab_size, dtype=np.int64)[:, None]).astype(np.float)
                feed_dict[train_labels[i]] = (np.arange(vocab_size + 1) == np.full(batch_size, vocab_size, dtype=np.int64)[:, None]).astype(np.float)
            
            # Train.
            _, l, lr= session.run([optimizer, loss, learning_rate], feed_dict=feed_dict)
            mean_loss += l
            
        if step % summary_frequency == 0:
            print('Step: %d, Learning Rate: %f, Average loss: %f' % (step, lr, mean_loss/(summary_frequency * max_seq_len)))
            mean_loss = 0
            
            # Create five sample sequences and see how well the model reverses them.
            print('       Input | Output')
            for j in range(5):
                reset_sample.run()
                
                # Create sequence.
                seq_len = np.random.randint(max_seq_len) + 1
                seq_in = np.append(np.random.randint(vocab_size, size=seq_len), np.full(max_seq_len - seq_len, vocab_size, dtype=np.int64))
                
                # Encode.
                for i in range(max_seq_len):
                    train_enc.run({sample_i: (np.arange(vocab_size + 1) == [[seq_in[i]]]).astype(np.float)})
                    
                # Prepare decoding.
                seq_out = np.ndarray(0, dtype=np.int64)
                feed = (np.arange(vocab_size + 1) == [[vocab_size]]).astype(np.float)
                init_dec.run()
                
                # Decode until EOS recieved, or sequence length exceeds 2 * max_seq_len.
                while len(seq_out) == 0 or (seq_out[-1] != vocab_size and len(seq_out) < 2 * max_seq_len):
                    prediction = session.run([sample_prediction], {sample_i: feed})
                    seq_out = np.append(seq_out, np.argmax(prediction))
                    feed = (np.arange(vocab_size + 1) == [[seq_out[-1]]]).astype(np.float)
                    
                # Trim input/output, print results.
                seq_in = seq_in[:seq_len]
                seq_out = seq_out[:-1]
                print('  ' * (max_seq_len - seq_len), seq_in, '|', seq_out)

Initialized
Step: 0, Learning Rate: 0.010000, Average loss: 0.002358
       Input | Output
   [0 9 9 1] | []
 [7 6 0 9 2] | []
       [8 7] | []
         [5] | []
   [0 1 1 5] | []
Step: 1000, Learning Rate: 0.010000, Average loss: 0.190882
       Input | Output
         [7] | [7]
 [6 5 9 0 7] | [7 0 9 5 6]
   [3 3 4 4] | [4 4 3 3]
 [2 4 2 8 9] | [9 8 2 4 2]
         [5] | [5]
Step: 2000, Learning Rate: 0.010000, Average loss: 0.011923
       Input | Output
   [1 5 6 6] | [6 6 5 1]
         [0] | [0]
       [5 6] | [6 5]
       [3 3] | [3 3]
 [8 0 0 6 9] | [9 6 0 0 8]
Step: 3000, Learning Rate: 0.010000, Average loss: 0.007677
       Input | Output
 [9 9 8 9 2] | [2 9 8 9 9]
   [9 1 2 7] | [7 2 1 9]
     [5 3 4] | [4 3 5]
       [5 3] | [3 5]
     [8 6 4] | [4 6 8]
Step: 4000, Learning Rate: 0.010000, Average loss: 0.005963
       Input | Output
         [8] | [8]
         [3] | [3]
   [7 0 0 0] | [0 0 0 7]
 [1 7 6 2 7] | [7 2 6 7 1]
     [2 4 6] | [6 4 2]
Step: 5000, Learning Rate: 0.